# Project 3: Topic Classification using Naive Bayes

**Solution**

# Intro
---
In this project, you'll work with text data from newsgroup posts on a variety of topics. You'll train classifiers to distinguish posts by topics inferred from the text. Whereas with digit classification, where each input is relatively **dense** (represented as a 28x28 matrix of pixels, many of which are non-zero), here each document is relatively **sparse** (represented as a **bag-of-words**). Only a few words of the total vocabulary are active in any given document. The assumption is that a label depends only on the count of words, not their order.

The `sklearn` documentation on feature extraction may be useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on Slack, but <b> please prepare your own write-up with your own code. </b>

## Grading
---
- Make sure to answer every part in every question.
 - There are 7 questions and one extra credit question. 
 - Read carefully what is asked including the notes.
 - Additional points may be deducted if:
   - the code is not clean and well commented, 
   - and if the functions or answers are too long.

 ## Requirements:
---
1. Comment your code.
1. All graphs should have titles, label for each axis, and if needed a legend. It should be understandable on its own.
1. All code must run on colab.research.google.com
1. You should not import any additional libraries.
1. Try and minimize the use of the global namespace (meaning keep things in functions).



In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

Load the data, stripping out metadata so that only textual features will be used, and restricting documents to 4 specific topics. By default, newsgroups data is split into training and test sets, but here the test set gets further split into development and test sets.  (If you remove the categories argument from the fetch function calls, you'd get documents from all 20 topics.)

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test  = fetch_20newsgroups(subset='test',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels   = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels     = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('dev label shape:',      dev_labels.shape)
print('test label shape:',     test_labels.shape)
print('labels names:',         newsgroups_train.target_names)

training label shape: (2034,)
dev label shape: (676,)
test label shape: (677,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


### Question 1: Examining your data
---

 1. For each of the first 5 training examples, print the text of the message along with the label (checkout newsgroups_train.target_names).

In [3]:
def Q1(num_examples=5):
    ### STUDENT START ###
    for ind in range(num_examples):
        print("\033[1m" + "***Message at index " + str(ind) + " has Label: " + 
              newsgroups_train.target_names[train_labels[ind]] + "***" + "\033[0m")
        print()    
        print(train_data[ind])
        print()
        print("-------End of message at index" + str(ind) + "-------")
        print()
    ### STUDENT END ###

Q1(5)

***Message at index 0 has Label: comp.graphics***

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

-------End of message at index0-------

***Message at index 1 has Label: talk.religion.misc***



Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nop

### Question 2: Text representation
---

1. Transform the training data into a matrix of **word** unigram feature vectors.
  1. What is the size of the vocabulary? 
  1. What is the average number of non-zero features per example?  
  1. What is the fraction of the non-zero entries in the matrix?  
  1. What are the 0th and last feature strings (in alphabetical order)?
  - _Use `CountVectorization` and its `.fit_transform` method.  Use `.nnz` and `.shape` attributes, and `.get_feature_names` method._
1. Now transform the training data into a matrix of **word** unigram feature vectors restricting to the vocabulary with these 4 words: ["atheism", "graphics", "space", "religion"].  Confirm the size of the vocabulary. 
  1. What is the average number of non-zero features per example?
  - _Use `CountVectorization(vocabulary=...)` and its `.transform` method._
1. Now transform the training data into a matrix of **character** bigram and trigram feature vectors.  
  1. What is the size of the vocabulary?
  - _Use `CountVectorization(analyzer=..., ngram_range=...)` and its `.fit_transform` method._
1. Now transform the training data into a matrix of **word** unigram feature vectors and prune words that appear in fewer than 10 documents.  
  1. What is the size of the vocabulary?<br/>
  - _Use `CountVectorization(min_df=...)` and its `.fit_transform` method._
1. Now again transform the training data into a matrix of **word** unigram feature vectors. 
 1. What is the fraction of words in the development vocabulary that is missing from the training vocabulary?
 - _Hint: Build vocabularies for both train and dev and look at the size of the difference._

Notes:
* `.fit_transform` makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").
* `.fit_transform` and `.transform` return sparse matrix objects.  See about them at http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. 

In [4]:
def Q2():
    ### STUDENT START ###
    vectorizer = CountVectorizer()
    X_features = vectorizer.fit_transform(train_data)
    
    # X_features has as many rows as the number of train_data entries and 
    # as many columns as the vocabulary size. Each row is sparse containig
    # a (small) subset of the vocabulary being the non-zero entry, and 
    # zero for words that don't exist in that document.
    # len(vectorizer.get_feature_names()) will also give the size of the vocabulary
    print("1A. Size of the vocabulary is", X_features.shape[1])
    print()
    
    cnt_non_zero_features = 0
    for i in range(X_features.shape[0]): # for as many rows in X_features
        cnt_non_zero_features += X_features[i].nnz # accumulate the non-zero features
        
    print("1B. Average non-zero feature per example is", 
          round(cnt_non_zero_features/X_features.shape[0], 3))
    print()
    
    print("1C. Average non-zero entires in the matrix is",
          round(X_features.nnz/(X_features.shape[0] * X_features.shape[1]), 4))
    print()
    
    print("1D. The 0th feature string is", vectorizer.get_feature_names()[0], 
         "and the last feature string is", vectorizer.get_feature_names()[-1])
    print()
    
    # Init the CountVectorizer with the restricted vocabulary
    vectorizer = CountVectorizer(vocabulary=["atheism", "graphics", "space", "religion"])
    X_features_vocab = vectorizer.fit_transform(train_data)
    
    cnt_non_zero_features = 0
    for i in range(X_features_vocab.shape[0]):
        cnt_non_zero_features += X_features_vocab[i].nnz
        
    print("Confirmation: size of the vocabulary with restriction is indeed", X_features_vocab.shape[1])
    print("2A. Average non-zero feature per example is", 
          round(cnt_non_zero_features/X_features_vocab.shape[0], 3))
    print()
    
    vectorizer = CountVectorizer(ngram_range=(2, 3), analyzer='char')
    X_features_bitri = vectorizer.fit_transform(train_data)
    print("3A. Size of the Vocabulary with bigram, trigram is", X_features_bitri.shape[1])
    print()
    
    vectorizer = CountVectorizer(min_df=10)
    X_features_min_df = vectorizer.fit_transform(train_data)
    print("4A. Size of vocabulary with word frequency >= 10 is", X_features_min_df.shape[1])
    print()
    
    vectorizer = CountVectorizer()
    X_features_dev = vectorizer.fit_transform(dev_data)
    print("5A. Fraction of words in DEV vocabulary that is missing from training vocabulary is",
          round((X_features.shape[1] - X_features_dev.shape[1])/X_features.shape[1], 4))
    ### STUDENT END ###

Q2()

1A. Size of the vocabulary is 26879

1B. Average non-zero feature per example is 96.706

1C. Average non-zero entires in the matrix is 0.0036

1D. The 0th feature string is 00 and the last feature string is zyxel

Confirmation: size of the vocabulary with restriction is indeed 4
2A. Average non-zero feature per example is 0.268

3A. Size of the Vocabulary with bigram, trigram is 35478

4A. Size of vocabulary with word frequency >= 10 is 3064

5A. Fraction of words in DEV vocabulary that is missing from training vocabulary is 0.3956


### Question 3: Initial model evaluation
---

1. Transform the training and development data to matrices of word unigram feature vectors.
1. Produce several k-Nearest Neigbors models by varying k, including one with k set to optimize f1 score.  For each model, show the k value and f1 score. 
1. Produce several Naive Bayes models by varying smoothing (alpha), including one with alpha set approximately to optimize f1 score.  For each model, show the alpha value and f1 score.
1. Produce several Logistic Regression models by varying L2 regularization strength (C), including one with C set approximately to optimize f1 score.  For each model, show the C value, f1 score, and sum of squared weights for each topic.
1. Why doesn't k-Nearest Neighbors work well for this problem?
1. Why doesn't Logistic Regression work as well as Naive Bayes does?
1. What is the relationship between logistic regression's sum of squared weights vs. C value?

Notes:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `CountVectorizer` and its `.fit_transform` and `.transform` methods to transform data.
* You can use `KNeighborsClassifier(...)` to produce a k-Nearest Neighbors model.
* You can use `MultinomialNB(...)` to produce a Naive Bayes model.
* You can use `LogisticRegression(C=..., solver="liblinear", multi_class="auto")` to produce a Logistic Regression model.
* You can use `LogisticRegression`'s `.coef_` method to get weights for each topic.
* You can use `metrics.f1_score(..., average="weighted")` to compute f1 score.

In [ ]:
def Q3():
    ### STUDENT START ###
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(train_data)
    X_dev = vectorizer.transform(dev_data)
    
    print("KNN F1 Score:")
    max_f1 = -1
    max_f1_neighbor = 1
    for i in range(1,10):
        classifier = KNeighborsClassifier(n_neighbors=(2*i-1))
        classifier.fit(X_train, train_labels)
        y_pred = classifier.predict(X_dev)
        f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
        if max_f1 == -1:
            max_f1 = f1
        else:
            if f1 > max_f1:
                max_f1 = f1
                max_f1_neighbor = 2*i - 1
        print("    Neighbors:", 2*i-1, "F1 Score:", f1)
    print("    ****Optimal F1 score is", max_f1, "with", max_f1_neighbor, "neighbors")
    print()
    
    print("MultinomialNB F1 Score:")
    max_f1 = -1
    max_f1_alpha = 0
    for i in [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0, 2.0]:
        clf = MultinomialNB(alpha=i)
        clf.fit(X_train, train_labels)
        y_pred = clf.predict(X_dev)
        f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
        if max_f1 == -1:
            max_f1 = f1
        else:
            if f1 > max_f1:
                max_f1 = f1
                max_f1_alpha = i
        print("    Alpha:", i, "F1 Score:", f1)
    print("    ****Optimal F1 score is", max_f1, "for alpha", max_f1_alpha)
    print()
    
    sorted_categories = [i for i in newsgroups_train.target_names] # get label names as a copy
        
    print("Logistic regression:")
    print("                        ------------------Sum of squares of weights---------------------")
    print("    C       ", "F1 Score", "  ", 
          sorted_categories[0] +'    '+ sorted_categories[1]+'    '+ 
          sorted_categories[2] +'    '+ sorted_categories[3])
    max_f1 = -1
    max_f1_C = 0
    W2_category0 = []
    W2_category1 = []
    W2_category2 = []
    W2_category3 = []
    c_val = [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0, 2.0]
    for c in c_val:
        clf = LogisticRegression(C=c, solver="liblinear", multi_class="auto")
        clf.fit(X_train, train_labels)
        y_pred = clf.predict(X_dev)
        f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
        if max_f1 == -1:
            max_f1 = f1
        else:
            if f1 > max_f1:
                max_f1 = f1
                max_f1_C = c
        wt_0 = np.sum(clf.coef_[0]**2)
        W2_category0.append(wt_0)
        wt_1 = np.sum(clf.coef_[1]**2)
        W2_category1.append(wt_1)
        wt_2 = np.sum(clf.coef_[2]**2)
        W2_category2.append(wt_2)
        wt_3 = np.sum(clf.coef_[3]**2)
        W2_category3.append(wt_3)
        print('  %1.3f ' % c, '     %1.4f  '% f1, '   %3.3f   '% round(wt_0, 3), 
              '       %3.3f   '% round(wt_1, 3), '      %3.3f    '% round(wt_2, 3), 
              '      %3.3f    '%  round(wt_3, 3))
    print("    ****Optimal F1 score is", max_f1, "for C =", max_f1_C)
    print()
    
    fig, ax = plt.subplots()
      
    ax.plot(c_val, W2_category0, label=sorted_categories[0])
    ax.plot(c_val, W2_category1, label=sorted_categories[1])
    ax.plot(c_val, W2_category2, label=sorted_categories[2])
    ax.plot(c_val, W2_category3, label=sorted_categories[3])
    
    ax.set_title("Sum of squares of weights Vs C Value")
    ax.set_xlabel('C value of logistic regression')
    ax.set_ylabel('Sum of square of weights')
    ax.legend()
   
    ### STUDENT END ###

Q3()

KNN F1 Score:
    Neighbors: 1 F1 Score: 0.3805
    Neighbors: 3 F1 Score: 0.4084
    Neighbors: 5 F1 Score: 0.4288
    Neighbors: 7 F1 Score: 0.4505
    Neighbors: 9 F1 Score: 0.4366
    Neighbors: 11 F1 Score: 0.4266
    Neighbors: 13 F1 Score: 0.424
    Neighbors: 15 F1 Score: 0.4326
    Neighbors: 17 F1 Score: 0.4486
    ****Optimal F1 score is 0.4505 with 7 neighbors

MultinomialNB F1 Score:
    Alpha: 0.001 F1 Score: 0.7703
    Alpha: 0.01 F1 Score: 0.7752
    Alpha: 0.1 F1 Score: 0.7903
    Alpha: 0.25 F1 Score: 0.789
    Alpha: 0.5 F1 Score: 0.7863
    Alpha: 0.75 F1 Score: 0.7845
    Alpha: 1.0 F1 Score: 0.7777
    Alpha: 2.0 F1 Score: 0.769
    ****Optimal F1 score is 0.7903 for alpha 0.1

Logistic regression:
                        ------------------Sum of squares of weights---------------------
    C        F1 Score    alt.atheism    comp.graphics    sci.space    talk.religion.misc
  0.001       0.6193      0.165           0.201          0.181           0.187    
  0.010  

ANSWER: 

***Questions 5 and 6***
The KNN model won't work so well. We've over 25K features. The idea behind KNN is to compare the item with K nearest neighbors, and pick the one type that has most match. In the case of documents this methods works only when the dev data conatins doc. that are nearly identical to one of the training data set entries. Given that each document contains only a small subset of the entire vocabulary (case of sparse matrix compared to a reasonable matrix used with hand-written digits pixel image) KNN isn't able to a great job of figuring out possible right neigbhors. In other words, if you go well beyond these neighbors there may be a better match. The sparsity of the array that represent each of the document doesn't help in KNN calculation.

The logistic regression does better than KNN, but not as good as MultinomialNB. For items like documents which trun into a sparse matrix when features are transformed into numbers, the concept of counting the number of each word in each document and comparing it with the training is a good way to match. Logistic regression is starting with a hyperplane and trying to tweak the intercept and slope of the hyperplane to maximize likelihood. This is a slower process compared to MultinomiaNB. It may also be due to the inherent different in generative model (Naive Bayes) Vs Discriminative Model (Logistic Regression).


***Question 7***
The sum of squares of weights increase with C value. While it starts a bit steeply the growth seems to slow down with larger C values. 

### Question 4: Feature exploration
---

1. Transform the data to a matrix of word **bigram** feature vectors.  Produce a Logistic Regression model.
1. For each topic, find the 5 features with the largest weights (not absolute value). If there are no overlaps, you can expect 20 features in total.
1. Show a 20 row (features) x 4 column (topics) table of the weights. So, for each of the features (words) found, we show their weight for all topics.
1. Do you see any surprising features in this table?

Notes:
* Train on the transformed training data.
* You can use `CountVectorizer` and its `.fit_transform` method to transform data.
* You can use `LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")` to produce a Logistic Regression model.
* You can use `LogisticRegression`'s `.coef_` method to get weights for each topic.
* You can use `np.argsort` to get indices sorted by element value. 


In [ ]:
def Q4():
    ### STUDENT START ###
    vectorizer = CountVectorizer(ngram_range=(2,2), analyzer='word')
    X_train = vectorizer.fit_transform(train_data)
    clf = LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")
    clf.fit(X_train, train_labels)
    feature_names = vectorizer.get_feature_names()

    sorted_categories = [i for i in newsgroups_train.target_names] # get label names as a copy
    
    dict_data = {'top_words': [], 'index': []} # init dict for storing metrics
    for i in sorted_categories:
        dict_data[i] = []
    
    print("For each topic the top 5 features with their repective coefficinets in descending order are:")
    for i in range(clf.coef_.shape[0]):
        sorted_array = np.sort(clf.coef_[i]) # sort the row (in asceding order)
        top_5_coef = sorted_array[-1:-6:-1] # extract the last 5 elements, starting from the largest
        top_5_features = []
        for j in range(5):
            ind = np.where(clf.coef_[i] == top_5_coef[j])[0][0] # extract the index in the coef array
            word = feature_names[ind] # extract the word corresponding to that index
            top_5_features.append(word) # append the word to the list
            dict_data['top_words'].append(word) # append the word to the corresponding key in the dict
            dict_data['index'].append(ind) # append the index to correspondig list
            dict_data[sorted_categories[i]].append(top_5_coef[j]) # append the coefficient
            for k in range(clf.coef_.shape[0]): # for all rows in the coef matrix
                if k != i: # if the row is not the same row that was sorted for top 5 coef identification
                    coef = clf.coef_[k][ind]
                    dict_data[sorted_categories[k]].append(coef) # append the coefficient           
        print("    Topic:", sorted_categories[i])
        for j in range(5):
            print("           ", top_5_features[j], ":%4f" % top_5_coef[j] )
        print()
        
    df_data = pd.DataFrame(dict_data) # convert to DF for ease of printing
    print(df_data)
    ### STUDENT END ###

Q4()

ANSWER: 
1. It's interesting to see that the phrase "the fbi" is the top contributor to the "talk.religion.misc" topic.
2. For all topics, except the word associated with that topic, coefficients associatd with other words are negative. There's one exception - for the topic "talk.religion.misc" the phrase "cheers kent" has a positive association.
3. The phrase "cheers kent" appears in both topics "alt.aethism" and "talk.religion.misc", and has coefficients which are quite close in values.

### Question 5: Pre-processing for text
---

To improve generalization, it is common to try preprocessing text in various ways before splitting into words. For example, you could try transforming strings to lower case, replacing sequences of numbers with single tokens, removing various non-letter characters, and shortening long words.

1. Produce a Logistic Regression model (with no preprocessing of text). **Note that you may need to override the "default" preprocessing with an identity function**. Evaluate and show its f1 score and size of the dictionary.
1. Produce an improved Logistic Regression model by preprocessing the text. Evaluate and show its f1 score and size of the vocabulary.  Aim for an improvement in f1 score of 0.02. **Note: this is actually very hard**.
1. How much did the improved model reduce the vocabulary size?

Notes:
* Things you can try: ** ???: Anything else we can suggest** 
 - Look at default pre-processing done.
 - Removing stop words.
 - Experiment with different ways of getting rid of apostrophe's such as replacing them with spaces or with empty strings.
  - Lower casing.
  - Including both lowercase and original case versions of a word.
  - nltk functions such as stemming.
* Train on the "transformed" training data, the data after you applied pre-processing.
* Evaluate on the transformed development data. Note that you never want to "learn" anything from the dev data.
* You can use `CountVectorizer(preprocessor=...)` to preprocess strings with your own custom-defined function.
* `CountVectorizer` default is to preprocess strings to lower case.
* You can use `LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")` to produce a logistic regression model.
* You can use `metrics.f1_score(..., average="weighted")` to compute f1 score.
* If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular.
* The order you apply pre-processing may produce different results.


In [ ]:
def Q5():
    ### STUDENT START ###   
    def no_preprocessor(text):
        return text
    
    def my_preprocessor(text):
        text = text.lower()
        
        text = re.sub("\\W", " ", text) # remove special characters with blank
        text = re.sub('[^A-Za-z0-9]', ' ', text) # remove non-alpha numerics
        text = re.sub('[/\r?\n|\r/]', ' ', text) # remove line-breaks
        text = re.sub("(\w{9})\w+", "\\1", text) # shorten longer words
        
        ps = nltk.stem.PorterStemmer() # init for stemmimg the word
        words = re.split("\\s+", text) # split text into words 
        stemmed_words = [ps.stem(word=word) for word in words] # stem the words
        
        return ' '.join(stemmed_words)
        
    vectorizer = CountVectorizer(preprocessor=no_preprocessor) # call with dummy preprocessor

    X_train = vectorizer.fit_transform(train_data)
    X_dev = vectorizer.transform(dev_data)
    
    clf = LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")    
    clf.fit(X_train, train_labels)
    y_pred = clf.predict(X_dev)
    f1_normal = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
    vocab_normal = X_train.shape[1]
    
    print("Logistic regression with no preprocessing:")
    print("         F1 sore is", f1_normal)
    print("         Size of the dictionary is", vocab_normal)
    print()
    
    vectorizer = CountVectorizer(preprocessor=my_preprocessor) # call with my_preprocessor
    X_train = vectorizer.fit_transform(train_data)
    
    feature_names = vectorizer.get_feature_names()
    
    X_dev = vectorizer.transform(dev_data)
    
    clf = LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")    
    clf.fit(X_train, train_labels)
    y_pred = clf.predict(X_dev)
    f1_preproc = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
    vocab_preproc = X_train.shape[1]
    
    print("Logistic regression with preprocessing:")
    print("         F1 sore is", f1_preproc)
    print("         Size of the dictionary is", vocab_preproc)
    print()
    
    print("With Pre-Processing...")
    print("      Vocabulary size reduced by", vocab_normal-vocab_preproc, "or", 
          round((vocab_normal-vocab_preproc)*100/vocab_normal, 3), "precent")
    print("      F1 score improved by", round(f1_preproc-f1_normal, 4))

    ### STUDENT END ###

Q5()

### Question 6: L1 and L2 regularization
---

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. Logistic regression seeks the set of weights that minimizes errors in the training data AND has a small total size. The default L2 regularization computes this size as the sum of the squared weights (as in Part 3 above). L1 regularization computes this size as the sum of the absolute values of the weights. Whereas L2 regularization makes all the weights relatively small, **L1 regularization drives many of the weights to 0, effectively removing unimportant features**. For this reason, we can use it as a way to do "feature selection".

1. For several L1 regularization strengths ...
  1. Produce a Logistic Regression model using the **L1** regularization strength.  Reduce the vocabulary to only those features that have at least one non-zero weight among the four categories.
  1. Produce a new Logistic Regression model using the reduced vocabulary . For this new model, use an **L2** regularization strength of 0.5.  
  1. Evaluate and show the L1 regularization strength, vocabulary size, and f1 score associated with the new model.
1. Show a plot of f1 score vs. log vocabulary size.  Each point corresponds to a specific L1 regularization strength used to reduce the vocabulary.
1. How does performance of the models based on reduced vocabularies compare to that of a model based on the full vocabulary?

Notes:
* No need to apply pre-processing from question 5.
* Train on the transformed (i.e. CountVectorizer) training data.
* Evaluate on the transformed development data (using the CountVectorizer instance you trained on the training data).
* You can use `LogisticRegression(..., penalty="l1")` to produce a logistic regression model using L1 regularization.
* You can use `LogisticRegression(..., penalty="l2")` to produce a logistic regression model using L2 regularization.
* You can use `LogisticRegression(..., tol=0.015)` to produce a logistic regression model using relaxed gradient descent convergence criteria.  The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).
* (solver="liblinear" might be needed for it not to crash)

In [ ]:
def Q6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(train_data)
    X_dev = vectorizer.transform(dev_data)
    feature_names = vectorizer.get_feature_names()
    
    tol_lr = 0.015 # tolerance value for logistic regression
     
    c_val = [0.05, 0.25, 0.5, 0.75, 1.0, 1.2, 1.3, 2.0, 3.0]

    f1_l1 = [] # init list for all f1 scores when pnealty='l1'
    f1_l2 = [] # init list for all f1 scores when pnealty='l2'
    vocab_l1 = [] # init list for vocabulary size for penalty='l1'
    vocab_l2 = [] # init list for vocabulary size for penalty='l2'
    
    for c in c_val:
        if c == 1.5: # fine-tune the number of iterations to allow convergence
            num_iter = 175
        else:
            num_iter = 100
        clf = LogisticRegression(penalty='l1', C=c, solver="liblinear", max_iter=num_iter,
                                 multi_class="auto", tol=tol_lr)    
        clf.fit(X_train, train_labels)
        y_pred = clf.predict(X_dev)
        f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
        f1_l1.append(f1)
        coef_matrix = clf.coef_
        
        vocab_l1.append(X_train.shape[1])

        deleted_feature = [] # features/words that got deleted
        deleted_ind = [] # 
        retain_list = []
        for i in range(coef_matrix.shape[1]): # for all columns in the coef matrix
            if np.count_nonzero(coef_matrix[:, i]) == 0: # check if all zero column
                deleted_ind.append(i) # accumalte all col. indices for 0 col. value
                deleted_feature.append(feature_names[i]) # accumlate the corresponding deleted feature
            else: # if not a zero column
                retain_list.append(i) # appedn the non-zero indices to the pruned list
        
        X_train_pruned = X_train[:, retain_list] # prune X_train, X_dev by removing columns
        X_dev_pruned = X_dev[:, retain_list]     # that are to all-0 cols in coef matrix
        
        clf = LogisticRegression(penalty='l2', C=c, solver="liblinear", max_iter=125, 
                                 multi_class="auto", tol=tol_lr)    
        clf.fit(X_train_pruned, train_labels)
        y_pred = clf.predict(X_dev_pruned)
        f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)
        f1_l2.append(f1)
        vocab_l2.append(X_train_pruned.shape[1])
    
    dict_data = {'C-Value': c_val, 'f1(l1_full_vocab)': f1_l1, 
                 'f1(l2_reduced_vocab)': f1_l2, 
                 'vocab_full': vocab_l1, 'vocab_pruned': vocab_l2}
    df_data = pd.DataFrame(dict_data)
    print(df_data)
    
    log_vocab_l2 = np.log(vocab_l2)
    
    plt.figure(figsize=(10, 8))
    plt.plot(log_vocab_l2, f1_l2, 'v')
    
    plt.title("F1 score Vs Log Vocabulary size, with C-Value noted above the score")
    plt.xlabel('Log value of vocabulary size')
    plt.ylabel('F1 Score')
    for ind, val in enumerate(c_val):
        plt.text(log_vocab_l2[ind], f1_l2[ind], val)
    plt.show()
    ### STUDENT END ###

Q6()

ANSWER: 
Clearly, the influence of the C-Value is significant. At the low-end of the pruned vocabulary the F1 score for the reduced vocabulary case is higher that the case of full vocabulary. Given our prune proceudre (of dropping all words which have 0 coefficients for all class labels), a low value of C has, in general bad performance, and given that we eliminated all those that have 0 coefficient value, the prunded model is doing a bit better. 

For all other C-values teh F1 score with full vocabulary is in the F1 score is in the 0.69 to 0.7 range, and the F1 score for the pruned vocabulary is between 0.67 and 0.69. Barring a couple of C values the full vocabulary has higher F1 score but it's not significantly different. Compared to the significant vocbulary size reduction the F1 score isn't going down a whole lot. At C=2.0 the difference between the two F1 scores is 0.004, and the pruned case is tiny bit better.

### Question 7: TfIdf
---
As you may recall [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) stands for *term frequency inverse document frequency* and is a way to assign a weight to each word or token signifying their importance for a document in a corpus (a collection of documents).

Produce a Logistic Regression model based on data represented in tf-idf form, with L2 regularization strength of 100.  Evaluate and show the f1 score.  How is `TfidfVectorizer` different than `CountVectorizer`?

1. How is `TfidfVectorizer` different than `CountVectorizer`?
1. Show the 3 documents with highest R ratio, where ...
  - $R\,ratio = maximum\,predicted\,probability \div predicted\,probability\,of\,correct\,label$
1. Explain what the R ratio describes.
1. What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

Note:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `TfidfVectorizer` and its `.fit_transform` method to transform data to tf-idf form.
* You can use `LogisticRegression(C=100, solver="liblinear", multi_class="auto")` to produce a logistic regression model.
* You can use `LogisticRegression`'s `.predict_proba` method to access predicted probabilities.

In [ ]:
def Q7():
    ### STUDENT START ###
    vectorizer = TfidfVectorizer()
    X_train_tf = vectorizer.fit_transform(train_data)
    X_dev_tf = vectorizer.transform(dev_data)
    
    clf = LogisticRegression(C=100, solver="liblinear", multi_class="auto")
    clf.fit(X_train_tf, train_labels)
    y_pred = clf.predict(X_dev_tf)
    f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)

    print("F1 score from tf-idf is", f1)
    print()
    
    class_prob = clf.predict_proba(X_dev_tf)
    # print(class_prob.shape, dev_labels.shape)
    # print(class_prob[0])
    # print(dev_labels[0])
    
    # The class_prob matrix is extracted for the linear regression has the shape (number of
    # dev labels) x (number of categories). Each row is the estimated probability for the 
    # predicted label for each of the four categories. The one with the highest probability
    # is chosen as the predicted label.
    
    r_ratios = [] # init list to hold the tuple of (ratio, index)
    for i in range(class_prob.shape[0]): # for all rows in the class_prob matrix
        max_prob = np.max(class_prob[i]) # get the max probability for the row
        label_prob = class_prob[i][dev_labels[i]] # probability in that row for the dev label
        r_ratios.append((max_prob/label_prob, i))
    
    r_ratios.sort(reverse=True)
    
    print("***Top 3 documents with the R Ratio, dev label, predicted label, and the document***")
    print()
    for i in range(3):
        ratio, ind = r_ratios[i]
        print("\033[1m"+"R Ratio:"+"\033[0m", round(ratio, 4), 
              "\033[1m"+"Dev Label:"+"\033[0m", newsgroups_train.target_names[dev_labels[ind]],
              "\033[1m"+"Predicted Label:"+"\033[0m", newsgroups_train.target_names[y_pred[ind]],
              "\033[1m"+"Index:"+"\033[0m", ind)
        print("\033[1m"+"The document:"+"\033[0m")
        print(dev_data[ind])
        print()    

    ### STUDENT END ###

Q7()

ANSWER: 

***Question 3***: What does R Ratio mean?
R-Ratio is the ratio of the max probability of each row of the predictor's probability matrix upon the probability of the label actually predicted. The maxtrix, in the specific case, has as many rows as the number of DEV Labels, and has 4 columns, one corresponding to each class.

The value in the position (i, j) is the probability that the predicted label value, correpsonding to the ith predicted value, being class j. The model examines each row, and whichever column in that row has the highest value (probability) the predicted label will be that class.

The ratio should be 1, if the ith predicted value and the ith DEV lable (which was used to predict) are identical. Otherwise, the R-Ratio (always >= 1) is an indication of how far we've strayed from the true value. If R is high it indicates that the model strongly believe in the predicted value (which has the highest probability) and such a belief is off from the truth by a huge value - being ratio of the (probaility) of predicted label to that of the actual label.

***Question 4***
When we look at the docs that have the highest R ratio (i.e model strongly believed in it, but the actual label class has a low probability per the model's fit) we see the following:
- Doc. 1 has both relegious and computer related items. This doc. is a candidate for misclassification. There seems to be words that have both significance in reltion and computers.
- Doc. 2 again has the same problem as the 1st one - it's a pretty shory doc., and has more computer related words than relegious words.
- Doc. 3 is indeed a small doc. Without correlation of words it's difficult to classify this doc.

As for improvements, one option is to exclude words that can two or more interpretations. Thus, if we remove these features we might see the model fit a bit more correctly. A 2nd option is to use the "stop_words" option in the TfidfVectorizer() call, with a stop word list set to ENGLISH_STOP_WORDS.


### Question 8 EXTRA CREDIT:
---
Produce a Logistic Regression model to implement your suggestion from Part 7.

In [ ]:
def Q8():
    
    def my_preprocessor(text):
        text = text.lower()
        
        text = re.sub("\\W", " ", text) # remove special characters with blank
        text = re.sub('[^A-Za-z0-9]', ' ', text) # remove non-alpha numerics
        text = re.sub('[/\r?\n|\r/]', ' ', text) # remove line-breaks
        text = re.sub("(\w{9})\w+", "\\1", text) # shorten longer words
        
        ps = nltk.stem.PorterStemmer() # init for stemmimg the word
        words = re.split("\\s+", text) # split text into words 
        stemmed_words = [ps.stem(word=word) for word in words] # stem the words
        
        #return ' '.join(stemmed_words)
        return text
    
    stop_word_list = ENGLISH_STOP_WORDS
    vectorizer = TfidfVectorizer(stop_words=stop_word_list)
    #vectorizer = TfidfVectorizer(stop_words=stop_word_list, preprocessor=my_preprocessor)
    X_train_tf = vectorizer.fit_transform(train_data)
    X_dev_tf = vectorizer.transform(dev_data)
    
    clf = LogisticRegression(C=100, solver="liblinear", multi_class="auto")
    clf.fit(X_train_tf, train_labels)
    y_pred = clf.predict(X_dev_tf)
    f1 = round(metrics.f1_score(dev_labels, y_pred, average='weighted'), 4)

    print("F1 score from tf-idf is", f1)
    print()
    
    class_prob = clf.predict_proba(X_dev_tf)
    
    r_ratios = [] # init list to hold the tuple of (ratio, index)
    for i in range(class_prob.shape[0]): # for all rows in the class_prob matrix
        max_prob = np.max(class_prob[i]) # get the max probability for the row
        label_prob = class_prob[i][dev_labels[i]] # probability in that row for the dev label
        r_ratios.append((max_prob/label_prob, i))
    
    r_ratios.sort(reverse=True)
    
    print("***Top 3 documents with the R Ratio, dev label, predicted label, and the document***")
    print()
    for i in range(3):
        ratio, ind = r_ratios[i]
        print("\033[1m"+"R Ratio:"+"\033[0m", round(ratio, 4), 
              "\033[1m"+"Dev Label:"+"\033[0m", newsgroups_train.target_names[dev_labels[ind]],
              "\033[1m"+"Predicted Label:"+"\033[0m", newsgroups_train.target_names[y_pred[ind]],
              "\033[1m"+"Index:"+"\033[0m", ind)
        print("\033[1m"+"The document:"+"\033[0m")
        print(dev_data[ind])
        print()    

Q8()

For Q8 the F1 score improved a tiny bit - about 1.7%. The R ratio for the worst 3 cases has a mixed result. The 1st one went up, while the R ratio for the next two went down. It's misclassified but the model seems to be doing better.